# Vergleichen verschiedener Embeddings 

In diesem Notebook werden die verschiedenen Embedding Algorithmen demonstriert.

### imports

In [1]:
import sys
sys.path.append('..')

from db_connect import db_get_df, db_save_df
import joblib
import json
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from Embedding_creation.embedding_creator_TF_IDF import calc_all_tf_idf, calculate_distances
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer

/Users/br/Projects/Bachelorarbeit/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## LLAMA 2 Embeddings

### Laden der Modelle

In [ ]:
model = AutoModel.from_pretrained('mesolitica/llama2-embedding-1b-8k', trust_remote_code = True)
tokenizer = AutoTokenizer.from_pretrained('mesolitica/llama2-embedding-1b-8k')

In [ ]:
df = db_get_df("transcript_sentences")

### Tokenisierung der Sätze

In [ ]:
input_ids = tokenizer(
    df["sentence"].to_list(), 
    return_tensors = 'pt',
    padding = True
)

In [ ]:
v = model.encode(input_ids).detach().numpy()
v.shape

### Speichern der Embeddings

In [ ]:
df["embedding_json"] = [json.dumps(model.encode(chunk_text).detach().numpy()) for chunk_text in tqdm(input_ids)]


### Vereinigen der Tables

In [ ]:
df = db_get_df()
len(df)

In [ ]:
table_id = list("bcdyefghij")
all_df = db_get_df("transcript_segments_llama_2_a")
for id in table_id:
    df_temp = db_get_df(f"transcript_segments_llama_2_{id}")
    # print(len(df_temp))
    print(df_temp.head(1).iloc[0, 0])
    all_df = pd.concat([all_df, df_temp])


In [ ]:
db_save_df(all_df, "transcript_segments_llama_2_all")

In [ ]:
all_df = db_get_df("transcript_segments_llama_2_all")

In [ ]:
df = db_get_df()

In [ ]:
json_strings = [json.dumps(row.tolist()) for index, row in all_df.iterrows()]
df["embedding_json"] = json_strings

In [ ]:
df.iloc[3]

In [ ]:
db_save_df(df, "transcript_segments_llama_2")

## TF-IDF für alle Daten

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
import os
from dotenv import load_dotenv

load_dotenv()
DATA_PATH = os.getenv("DATA_PATH")

tf-idf auf lemmatisierte Sätze

In [ ]:
df = db_get_df(table="sentences_lemmatized")
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['sentence_lemmatized'])
tfidf_array = tfidf_matrix.toarray()

joblib.dump(tfidf_vectorizer, os.path.join(DATA_PATH,'matricies/tfidf_vectorizer200k.pkl'))
print("vectorizer file dumped")

In [ ]:
df = db_get_df(table="sentences_lemmatized")
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['sentence'])

### Abspeichern

In [ ]:
matrix_path = os.path.join(DATA_PATH,'matricies/tf_idf_matrix200k.npz')
sparse.save_npz(matrix_path, tfidf_matrix)

### Laden der Modelle

In [ ]:
matrix_path = os.path.join(DATA_PATH,'matricies/tf_idf_matrix230k.npz')
vectorizer_path = os.path.join(DATA_PATH,'matricies/tfidf_vectorizer230k.pkl')
tf_idf_matrix = sparse.load_npz(matrix_path)
tfidf_vectorizer = joblib.load(vectorizer_path)

### Optional speichern des Vocabulars

In [ ]:
sorted_dict = dict(sorted(tfidf_vectorizer.get_vocab().items()))
output_file = 'vocabulary.txt'
with open(output_file, 'w') as file:
    for key in sorted_dict.keys():
        file.write(key + '\n')


### Demonstration

In [ ]:
calc_all_tf_idf()

In [ ]:
from scripts.Embedding_creation.embedding_creator_TF_IDF import calc_all_tf_idf, calculate_distances


df_tfidf =  calculate_distances("Geschichte von Deutschland")

### IDF Analyse 

In [ ]:
from scipy import sparse
import joblib
import spacy

tf_idf_matrix = sparse.load_npz("tf_idf_matrix.npz")
tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')
feature_names = tfidf_vectorizer.get_feature_names_out()
idf_values = tfidf_vectorizer.idf_
idf_dict = dict(zip(feature_names, idf_values))
nlp = spacy.load("de_core_news_md")

def lemmatize_german_sentence(input_sentence, nlp):
    doc = nlp(input_sentence)
    lemmatized_words = []
    for token in doc:
        lemma = token.lemma_
        if lemma:
            lemmatized_words.append(lemma)
        else: 
            lemmatized_words.append(token)
    return lemmatized_words

In [ ]:
df = db_get_df("sentences_lemmatized")

In [ ]:
sentence = "Wer ist frau meier"
sentence = lemmatize_german_sentence(sentence, nlp)

encoded_words = [(idf_dict[word.lower()], word) for word in sentence if word.lower() in idf_dict]
print(encoded_words)

## Sentence Transformer

SBert
synchron - asynchron

Demonstration

In [1]:
import sys
sys.path.append('..')

from db_connect import db_get_df, db_save_df, save_pkl, load_pkl
from sentence_transformers import SentenceTransformer

/Users/br/Projects/Bachelorarbeit/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer('all-MiniLM-L6-v2')
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']


embeddings = model.encode(sentences)
save_pkl(sentences, embeddings, "test.pkl")

In [2]:
sentences, embeddings = load_pkl("MINI_L6_embeddings.pkl")

UnpicklingError: pickle data was truncated

[[-0.01371735 -0.04285153 -0.01562862 ...  0.10017824  0.12365722
  -0.0422967 ]
 [ 0.05645244  0.0550024   0.03137964 ...  0.06650873  0.08491519
  -0.03328491]
 [ 0.04393354  0.05893443  0.04817836 ...  0.0521628   0.05610655
   0.10206389]]
